## Dataload

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.models as models
from torch.utils.data import random_split

import torch.optim as optim
import os

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
osdir = '101_ObjectCategories'

transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                transforms.RandomVerticalFlip(),
                                transforms.Resize((224,224)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))])


dataset = datasets.ImageFolder(osdir,transform=transform)
dataset

In [ ]:
dataset_size = {'train': 7000, 'test': 2144}

traindata, testdata = random_split(dataset, [dataset_size['train'], dataset_size['test']])


In [ ]:

trainloader =DataLoader(traindata, 
                         batch_size=4,
                         shuffle=True, 
                         drop_last=False)

testloader = DataLoader(testdata, 
                        batch_size=4,
                        shuffle=False,
                        drop_last=False)

In [ ]:
len(trainloader)

## Train

In [ ]:
trainiter = iter(trainloader)
images, labels = trainiter.next()

In [ ]:
resnet18 = models.resnet18()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet18.parameters(),lr=0.001)

In [ ]:
y = resnet18(images)

In [ ]:
from tqdm import tqdm

epochs = 10 # dataset을 여러번 사용해 트레이닝을 시킵니다.

for epoch in tqdm(range(epochs)):
    running_loss = 0.0

    for i, data in enumerate(trainloader):
        # get the inputs
        inputs, labels = data
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = resnet18(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        step = i + 1
        running_loss += loss.item()
        
        if step % 250 == 0 and step != 0:    # print every 20 mini-batches
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' %
                  (epoch + 1, epochs, step, len(trainloader)//250* 250, running_loss / 250))
            running_loss = 0.0

## Test

In [ ]:
class_correct = list(0. for i in range(102))
class_total = list(0. for i in range(102))

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1) # prediction
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(102):
    print('Accuracy of %5s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))